In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

from models.seir import SEIRHD, SEIRHD_Severity

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, create_decile_csv
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.uncertainty import MCUncertainty
from utils.create_report import save_dict_and_create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from utils.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

import yaml

In [ ]:
predictions_dict = {}

In [ ]:
with open('../../configs/seir/default.yaml') as configfile:
    config = yaml.load(configfile, Loader=yaml.SafeLoader)

In [ ]:
for key in config['fitting']['data'].keys():
    if config['fitting']['data'][key] == 'None':
        config['fitting']['data'][key] = None 

In [ ]:
config['fitting']

## Params

In [ ]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

model=SEIRHD_Severity
forecast_days = 45

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)

num_trials_to_plot = 10
plot_topk_trials_for_columns = [Columns.confirmed, Columns.active, Columns.symptomatic, Columns.asymptomatic]
plot_ptiles_for_columns = [Columns.confirmed, Columns.active, Columns.symptomatic, Columns.asymptomatic]
sort_trials_by_column = Columns.confirmed
date_of_sorting_trials = '2020-07-31'

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.15),
    'T_inc': (4, 5),
    'T_inf': (3, 4),
    'T_recov_mild': (20, 30),
    'T_recov_moderate': (20, 30),
    'T_recov_severe': (0, 40),
    'P_moderate': (0, 0.5),
    'P_severe': (0, 0.1),
    'P_fatal': (0, 0.1),
    'E_hosp_ratio': (0, 2),
    'I_hosp_ratio': (0, 1)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) #Smoothing
    
predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m1']['data_last_date']

In [ ]:
single_fitting_cycle.__code__.co_varnames

In [ ]:
predictions_dict['m1']['best_params']

In [ ]:
predictions_dict['m2']['best_params']

## Create Master Loss Dataframe

### M1 Loss DataFrame

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss DataFrame

In [ ]:
predictions_dict['m2']['df_loss']

## Sensitivity Plot

In [ ]:
var_tuples = [
    ('lockdown_R0', np.linspace(1, 1.5, 101), 'total', None),
    ('I_hosp_ratio', np.linspace(0, 1, 201), 'total', None),
    ('E_hosp_ratio', np.linspace(0, 2, 201), 'total', None),
    ('P_moderate', np.linspace(0, 0.5, 201), 'symptomatic', 'total'),
    ('P_severe', np.linspace(0, 0.5, 201), 'critical', 'total'),
    ('P_fatal', np.linspace(0, 0.5, 201), 'deceased', 'total'),
    ('T_recov_mild', np.linspace(1, 100, 101), 'asymptomatic', 'total'),
    ('T_recov_moderate', np.linspace(1, 100, 101), 'symptomatic', 'total'),
    ('T_recov_severe', np.linspace(1, 100, 101), 'critical', 'total'),
    ('T_recov_fatal', np.linspace(1, 100, 101), 'deceased', 'total')
]

predictions_dict['m1']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m1', var_tuples=var_tuples)
predictions_dict['m2']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m2', var_tuples=var_tuples)

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecasts'] = {}
predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m2', model=model, days=forecast_days)
predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict, (state, district), error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1', model=model, forecast_days=forecast_days)
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed, column=sort_trials_by_column)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2', model=model, forecast_days=forecast_days)
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed, column=sort_trials_by_column)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=num_trials_to_plot,
    which_compartments=plot_topk_trials_for_columns
predictions_dict['m2']['plots']['forecasts_topk'] = {}
for column in plot_topk_trials_for_columns:
    predictions_dict['m2']['plots']['forecasts_topk'][column.name] = kforecasts[column]

## Uncertainty + Uncertainty Forecasts

In [ ]:
uncertainty = MCUncertainty(predictions_dict, date_of_sorting_trials)

In [ ]:
uncertainty.beta_loss

In [ ]:
uncertainty_forecasts = uncertainty.get_forecasts()
for key in uncertainty_forecasts.keys():
    predictions_dict['m2']['forecasts'][key] = uncertainty_forecasts[key]['df_prediction']

In [ ]:
predictions_dict['m2']['beta'] = uncertainty.beta
predictions_dict['m2']['beta_loss'] = uncertainty.beta_loss
predictions_dict['m2']['deciles'] = uncertainty_forecasts

In [ ]:
predictions_dict['m2']['plots']['forecast_best_50'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 50], error_bars=False)
predictions_dict['m2']['plots']['forecast_best_80'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 80], error_bars=False)

In [ ]:
ptiles_plots = plot_ptiles(predictions_dict, which_compartments=plot_ptiles_for_columns)
predictions_dict['m2']['plots']['forecasts_ptiles'] = {}
for column in plot_ptiles_for_columns:
    predictions_dict['m2']['plots']['forecasts_ptiles'][column.name] = ptiles_plots[column]

## Create Report

In [ ]:
save_dict_and_create_report(predictions_dict, ROOT_DIR=output_folder)

In [ ]:
log_wandb(predictions_dict)